# Goal

<h3 style="color:blue">assess the quality of summaries written by students</h3>
<h3 style="color:indigo">evaluate how well a student represents the main idea and details of a source text, as well as the clarity, precision, and fluency of the language used in the summary</h3>
<h3 style="color:red">Freely & publicly available external data is <b>allowed</b>, including pre-trained models</h3>
<h3>This is Multi-Output problem</h3>

### Use Hugging Face Library
### Use NLTK
### Use Tensorflow

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import math
import subprocess
from tqdm import tqdm
import pickle

In [3]:
import tensorflow as tf

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, explained_variance_score, median_absolute_error

In [5]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [6]:
import transformers
from transformers import AutoTokenizer, TFBertModel

In [7]:
prompts_train = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv')
summaries_train = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv')
prompts_test = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv')
summaries_test = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv')

In [8]:
train = pd.merge(prompts_train, summaries_train, on='prompt_id')
test = pd.merge(prompts_test, summaries_test, on='prompt_id')

In [9]:
train.rename(columns = {'text' : 'summary'}, inplace=True)
test.rename(columns = {'text' : 'summary'}, inplace=True)

In [10]:
train.head(2)

,prompt_id,prompt_question,prompt_title,prompt_text,student_id,summary,content,wording
0,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,00791789cc1f,1 element of an ideal tragedy is that it shoul...,-0.210614,-0.471415
1,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,0086ef22de8f,The three elements of an ideal tragedy are: H...,-0.970237,-0.417058


In [15]:
columns_needed = ["prompt_text", "summary"]
train_data = train[columns_needed]
test_data = test[columns_needed]

In [11]:
def preprocess_text(text):
    # Tokenize the text using spaCy
    doc = nlp(text)
    
    preprocessed_words = [token.lemma_.lower().strip() for token in doc if not token.is_stop and token.is_alpha]
    
    preprocessed_text = ' '.join(preprocessed_words)
    
    return preprocessed_text



In [12]:
tqdm.pandas()

In [16]:
train_data[['preprocessed_prompt_text', 'preprocessed_summary']] = train_data.progress_apply(lambda row: row.apply(preprocess_text), axis=1)

100%|██████████| 7165/7165 [17:47<00:00,  6.71it/s]


In [23]:
test_data[['preprocessed_prompt_text', 'preprocessed_summary']] = test_data.progress_apply(lambda row: row.apply(preprocess_text), axis=1)

100%|██████████| 4/4 [00:00<00:00, 72.51it/s]


In [30]:
from transformers import AutoTokenizer, TFAutoModel
model = TFAutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at sentence-transformers/all-MiniLM-L6-v2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


### Next time use prepare_tf_dataset which is used to directly tokenize and data colat and
### make dataset compatible with tensorflow
####       https://huggingface.co/docs/transformers/v4.31.0/en/main_classes/model#transformers.TFPreTrainedModel.prepare_tf_dataset

In [38]:

def vectorize_dataframe(dataframe, col):
    vectors = []
    for text in tqdm(dataframe[col].tolist()):
        text_tokens = tokenizer(text, return_tensors="tf", padding=True, truncation=True)
        
        output = model(**text_tokens)
        array = np.array(output.pooler_output)
        vectors.append( array.flatten() )
        
    return vectors
    

In [41]:
train_data['prompt_text_embedded'] = vectorize_dataframe(train_data, 'preprocessed_prompt_text')

100%|██████████| 7165/7165 [11:54<00:00, 10.03it/s]


In [42]:
train_data['summary_embedded'] = vectorize_dataframe(train_data, 'preprocessed_summary')

100%|██████████| 7165/7165 [11:47<00:00, 10.12it/s]


In [39]:
test_data['prompt_text_embedded'] = vectorize_dataframe(test_data, 'preprocessed_prompt_text')
test_data['summary_embedded'] = vectorize_dataframe(test_data, 'preprocessed_summary')

100%|██████████| 4/4 [00:00<00:00, 10.20it/s]


In [55]:
with open("/kaggle/working/sentence_transformer_prompt_text_embeddings.pkl", "wb") as file:
    pickle.dump(train_data['prompt_text_embedded'], file)
    
with open("/kaggle/working/sentence_transformer_summary_embeddings.pkl", "wb") as file:
    pickle.dump(train_data['summary_embedded'], file)
    
#with open("/kaggle/input/embeddings/BERT_summary_embeddings.pkl", "rb") as file:
 #   train_data['summary_embedded'] = pickle.load(file)

In [56]:
traning_set = train_data[['prompt_text_embedded', 'summary_embedded']]
testing_set = test_data[['prompt_text_embedded', 'summary_embedded']]

### Take average of embeddings  [Not required, just checking]

In [57]:
target1 = np.array(train['content'])
target1 = target1.astype('float32')

target2 = np.array(train['wording'])
target2 = target2.astype('float32')

#target = (target1, target2)

In [58]:
def convert_tensor_to_numpy(tensor):
        return np.array(tensor, dtype='float32')

traning_set = traning_set.applymap(convert_tensor_to_numpy)
testing_set = testing_set.applymap(convert_tensor_to_numpy)

In [59]:
def prepare_dataset(dataset):
    # Flatten the nested arrays in the DataFrame
    dataset['prompt_text_embedded'] = dataset['prompt_text_embedded'].apply(lambda x: x.flatten())
    dataset['summary_embedded'] = dataset['summary_embedded'].apply(lambda x: x.flatten())
    
    feature1 = np.array(dataset['prompt_text_embedded'].tolist())
    feature2 = np.array(dataset['summary_embedded'].tolist())
    
    features = np.concatenate((feature1, feature2), axis=1)
    
    return features

In [60]:
features = prepare_dataset(traning_set)

In [61]:
features_for_test = prepare_dataset(testing_set)

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [62]:
from tensorflow.keras.layers import Dense, Input, Flatten

In [63]:
# Define the model architecture
model_content = tf.keras.Sequential([
      tf.keras.layers.Dense(356, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(232, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(94, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(1, activation='linear')
])

model_wording = tf.keras.Sequential([
     tf.keras.layers.Dense(356, activation='relu'),
     tf.keras.layers.Dropout(0.2),
     tf.keras.layers.Dense(232, activation='relu'),
     tf.keras.layers.Dropout(0.2),
     tf.keras.layers.Dense(94, activation='relu'),
     tf.keras.layers.Dropout(0.2),
     tf.keras.layers.Dense(1, activation='linear')
])

In [64]:
model_content.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mse'])
model_wording.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mse'])

In [66]:
# Train your model using model.fit()
history1 = model_content.fit(features, target1, epochs=200, batch_size=25, validation_split=0.2)

Epoch 1/200
230/230 [==============================] - 3s 5ms/step - loss: 0.7477 - mae: 0.6742 - mse: 0.7477 - val_loss: 0.4621 - val_mae: 0.5478 - val_mse: 0.4621
Epoch 2/200
230/230 [==============================] - 1s 4ms/step - loss: 0.4889 - mae: 0.5464 - mse: 0.4889 - val_loss: 0.4123 - val_mae: 0.5163 - val_mse: 0.4123
Epoch 3/200
230/230 [==============================] - 1s 4ms/step - loss: 0.4440 - mae: 0.5245 - mse: 0.4440 - val_loss: 0.4546 - val_mae: 0.5456 - val_mse: 0.4546
Epoch 4/200
230/230 [==============================] - 1s 4ms/step - loss: 0.4122 - mae: 0.5057 - mse: 0.4122 - val_loss: 0.3878 - val_mae: 0.5029 - val_mse: 0.3878
Epoch 5/200
230/230 [==============================] - 1s 4ms/step - loss: 0.3926 - mae: 0.4938 - mse: 0.3926 - val_loss: 0.3644 - val_mae: 0.4854 - val_mse: 0.3644
Epoch 6/200
230/230 [==============================] - 1s 4ms/step - loss: 0.4005 - mae: 0.4985 - mse: 0.4005 - val_loss: 0.3779 - val_mae: 0.4954 - val_mse: 0.3779
Epoch 7/20

In [67]:
history2 = model_wording.fit(features, target2, epochs=200,batch_size=25, validation_split=0.2)

Epoch 1/200
230/230 [==============================] - 1s 4ms/step - loss: 0.6425 - mae: 0.6410 - mse: 0.6425 - val_loss: 0.5447 - val_mae: 0.5947 - val_mse: 0.5447
Epoch 2/200
230/230 [==============================] - 1s 4ms/step - loss: 0.6352 - mae: 0.6356 - mse: 0.6352 - val_loss: 0.5810 - val_mae: 0.6211 - val_mse: 0.5810
Epoch 3/200
230/230 [==============================] - 1s 5ms/step - loss: 0.6232 - mae: 0.6284 - mse: 0.6232 - val_loss: 0.5705 - val_mae: 0.6090 - val_mse: 0.5705
Epoch 4/200
230/230 [==============================] - 1s 4ms/step - loss: 0.5997 - mae: 0.6226 - mse: 0.5997 - val_loss: 0.6776 - val_mae: 0.6529 - val_mse: 0.6776
Epoch 5/200
230/230 [==============================] - 1s 4ms/step - loss: 0.6120 - mae: 0.6239 - mse: 0.6120 - val_loss: 0.4973 - val_mae: 0.5719 - val_mse: 0.4973
Epoch 6/200
230/230 [==============================] - 1s 4ms/step - loss: 0.5987 - mae: 0.6184 - mse: 0.5987 - val_loss: 0.5681 - val_mae: 0.6027 - val_mse: 0.5681
Epoch 7/20

In [68]:
evaluate_on_train_content = model_content.evaluate(features, target1)
evaluate_on_train_wording = model_wording.evaluate(features, target2)

224/224 [==============================] - 1s 2ms/step - loss: 0.2547 - mae: 0.3791 - mse: 0.2547


In [69]:
print('evaluate_on_train_content', evaluate_on_train_content)
print('evaluate_on_train_wording',evaluate_on_train_wording)

evaluate_on_train_content [0.15120674669742584, 0.2921070456504822, 0.15120674669742584]
evaluate_on_train_wording [0.2547479569911957, 0.3790515065193176, 0.2547479569911957]


In [70]:
content_prediction = model_content.predict(features)
wording_prediction = model_wording.predict(features)

224/224 [==============================] - 0s 2ms/step


### Predict on test

In [71]:
test_pred_content = model_content.predict(features_for_test)
test_pred_wording = model_wording.predict(features_for_test)

1/1 [==============================] - 0s 23ms/step


## submission

In [72]:
test_pred_content = test_pred_content.reshape(-1)
test_pred_wording = test_pred_wording.reshape(-1)

In [73]:
submission = pd.DataFrame({
    'student_id' : test['student_id'],
    'content' : test_pred_content,
    'wording' : test_pred_wording
})

In [74]:
submission.to_csv('submission.csv', index=False)

In [75]:
submission.head()

,student_id,content,wording
0,000000ffffff,-1.156206,-1.089189
1,222222cccccc,-1.156206,-1.089189
2,111111eeeeee,-1.156206,-1.089189
3,333333dddddd,-1.156206,-1.089189
